In [3]:
import helper_functions as hf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn




In [4]:
df = hf.import_csv_data("./clustering_data/bg_imputed_onehot_cols.csv")
df




,PcrKey,Age,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSystemResponseTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin,EMSTransportTimeMin,EMSTotalCallTimeMin,...,NasemsoRegion_Great Lakes,NasemsoRegion_South,NasemsoRegion_West,NasemsoRegion_Western Plains,EndOfEMS_Expired in ED,EndOfEMS_Expired in the Field,EndOfEMS_Ongoing Resuscitation in ED,EndOfEMS_ROSC in the Field,EndOfEMS_ROSC in the ED,EndOfEMS_Ongoing Resuscitation by Other EMS
0,71121582,23,11.000000,5.516667,34.933333,29.416667,25.556523,0.000000,14.676967,99.466667,...,0,1,0,0,0,1,0,0,0,0
1,71122461,120,0.000000,0.750000,4.133333,3.383333,18.316667,2.000000,6.733333,81.916667,...,0,1,0,0,0,0,1,0,0,0
2,71122902,63,0.000000,1.100000,8.683333,7.583333,34.650000,0.650000,15.416667,104.033333,...,0,1,0,0,1,0,0,0,0,0
3,71123389,92,114.000000,3.683333,7.366667,3.683333,14.083333,0.966667,4.212406,21.450000,...,0,1,0,0,0,1,0,0,0,0
4,71124833,48,88.760114,0.016667,9.133333,9.116667,23.081620,1.566667,8.590389,58.883333,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358858,131801464,57,0.000000,3.000000,8.000000,5.000000,13.000000,0.000000,2.612451,21.000000,...,1,0,0,0,0,1,0,0,0,0
358859,131801585,61,0.000000,6.000000,12.000000,6.000000,5.000000,0.000000,27.000000,119.000000,...,1,0,0,0,0,0,1,0,0,0
358860,131801624,92,0.000000,3.000000,5.000000,2.000000,32.000000,1.000000,2.860706,37.000000,...,1,0,0,0,0,1,0,0,0,0
358861,131801707,85,0.000000,6.000000,12.000000,6.000000,9.000000,2.000000,42.000000,146.000000,...,1,0,0,0,1,0,0,0,0,0


In [5]:
df_map = {}



In [6]:
# add feature: eArrest_03 - Resuscitation Attempted By EMS (file: FACTPCRARRESTRESUSCITATION_CA.csv)
df_ResuscitationAttempted = hf.import_csv_data("processed_data_cardiac_arrest/FACTPCRARRESTRESUSCITATION_CA.csv")
df_ResuscitationAttempted = df_ResuscitationAttempted[~df_ResuscitationAttempted["eArrest_03"].isin([7701001, 7701003])]
df_map['ResuscitationAttempted'] = df_ResuscitationAttempted.rename(columns = {"eArrest_03":"ResuscitationAttempted"})



In [7]:
df_map['ResuscitationAttempted']





,PcrKey,ResuscitationAttempted
0,71121582,3003007
1,71122461,3003001
2,71122461,3003005
3,71122902,3003001
4,71122902,3003003
...,...,...
693530,131801585,3003005
693531,131801624,3003007
693532,131801707,3003003
693533,131801707,3003005


In [8]:
# add feature list1: 
# eArrest_07_AEDUse_11_ArrestRhythm_16_CPRReason.csv - eArrest_07, 11, 16
df_multiple_treatments = hf.import_csv_data("processed_data_cardiac_arrest/eArrest_07_AEDUse_11_ArrestRhythm_16_CPRReason.csv")

feature_name_list1 = {"eArrest_07":"PriorUseOfAED","eArrest_11":"ArrestRhythm", "eArrest_16":"ReasonToStopCPR"}
for label, column_name in feature_name_list1.items():
    df_map[column_name] = df_multiple_treatments[["PcrKey"] + [label]]
    df_map[column_name] = df_map[column_name][~df_map[column_name][label].isin([7701001, 7701003, '7701001', '7701003'])]
    df_map[column_name].rename(columns = {label:column_name}, inplace=True)



In [9]:
# View the dictionary {column_name:column_df} for subsequent operations

# df_multiple_treatments

for label, column_name in feature_name_list1.items():
    print(df_map[column_name])




           PcrKey  PriorUseOfAED
1        71121582        3007001
2        71122461        3007001
3        71122902        3007001
4        71123389        3007001
5        71124833        3007001
...           ...            ...
448679  131801464        3007001
448680  131801585        3007005
448681  131801624        3007001
448682  131801707        3007003
448683  131801811        3007003

[412845 rows x 2 columns]
           PcrKey ArrestRhythm
1        71121582      3011001
2        71122461      3011001
3        71122902      3011005
4        71123389      3011001
5        71124833      3011001
...           ...          ...
448679  131801464      3011001
448680  131801585      3011001
448681  131801624      3011001
448682  131801707      3011007
448683  131801811      3011007

[370994 rows x 2 columns]
           PcrKey  ReasonToStopCPR
1        71121582          3016005
5        71124833          3016003
6        71145732          3016011
9        71146589          3016003
12 

In [10]:
# add feature list2 eProtocols: 
# FACTPCRPROTOCOL_CA.csv - eProtocols_01, eProtocols_02
df_protocol = hf.import_csv_data("processed_data_cardiac_arrest/FACTPCRPROTOCOL_CA.csv")
print("Length of df_protocol: ", df_protocol.shape[0])
relevant_protocols = [9914001, 9914003, 9914005, 9914007, 9914009, 9914075, 9914169, 9914171, 9914189, 9914201]
df_protocol = df_protocol[df_protocol["eProtocol_01"].isin(relevant_protocols)]

feature_name_list2 = {"eProtocol_01":"ProtocolsUsed"}
for label, column_name in feature_name_list2.items():
    df_map[column_name] = df_protocol[["PcrKey"] + [label]]
    df_map[column_name].rename(columns = {label:column_name}, inplace=True)




Length of df_protocol:  472853


/tmp/ipykernel_17834/1769437279.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_map[column_name].rename(columns = {label:column_name}, inplace=True)


In [11]:
# df_protocol
df_map["ProtocolsUsed"]




,PcrKey,ProtocolsUsed
17545,105499521,9914001
17546,114837636,9914001
17551,104465955,9914001
17553,102428919,9914003
17554,130355723,9914009
...,...,...
443561,116521903,9914171
443562,122412605,9914171
443563,118296478,9914171
443564,118011564,9914171


In [12]:
# add single feature - Witness: 
# FACTPCRARRESTWITNESS_CA.csv - eArrest_04 - Arrest Witnessed By
df_witness = hf.import_csv_data("processed_data_cardiac_arrest/FACTPCRARRESTWITNESS_CA.csv")
df_witness = df_witness[~df_witness["eArrest_04"].isin([7701001, 7701003, '7701001', '7701003'])]
df_map['ArrestWitnessedBy'] = df_witness.rename(columns = {"eArrest_04":"ArrestWitnessedBy"})




In [13]:
df_map['ArrestWitnessedBy']




,PcrKey,ArrestWitnessedBy
0,71121582,3004007
1,71122461,3004001
2,71122902,3004001
3,71123389,3004001
4,71124833,3004001
...,...,...
423044,131801464,3004001
423045,131801585,3004001
423046,131801624,3004001
423047,131801707,3004005


In [14]:
# add single feature - CPR Provided: 
# FACTPCRARRESTCPRPROVIDED_CA.csv - eArrest_09 - Type of CPR Provided
df_cpr_provided = hf.import_csv_data("processed_data_cardiac_arrest/FACTPCRARRESTCPRPROVIDED_CA.csv")
df_cpr_provided = df_cpr_provided[~df_cpr_provided["eArrest_09"].isin([7701001, 7701003, '7701001', '7701003'])]
df_map['TypeOfCPRProvided'] = df_cpr_provided.rename(columns = {"eArrest_09":"TypeOfCPRProvided"})





In [15]:
df_map['TypeOfCPRProvided']




,PcrKey,TypeOfCPRProvided
0,99983399,3009001
1,100021481,3009001
2,100012160,3009001
3,100026212,3009001
4,99995765,3009001
...,...,...
538661,130729416,3009019
538662,130823211,3009019
538663,131579073,3009019
538664,131430475,3009019


In [16]:
# add single feature - Cardiac Rhythm on Arrival at Destination: 
# FACTPCRARRESTRHYTHMDESTINATION_CA.csv - Arrest_17 - Cardiac Rhythm on Arrival at Destination (after EMS)
df_dest_rhythm = hf.import_csv_data("processed_data_cardiac_arrest/FACTPCRARRESTRHYTHMDESTINATION_CA.csv")
df_dest_rhythm = df_dest_rhythm[~df_dest_rhythm["eArrest_17"].isin([7701001, 7701003, '7701001', '7701003'])]
df_map['CardiacRhythmOnArrivalAtDestination'] = df_dest_rhythm.rename(columns = {"eArrest_17":"CardiacRhythmOnArrivalAtDestination"})





In [17]:
df_map['CardiacRhythmOnArrivalAtDestination']




,PcrKey,CardiacRhythmOnArrivalAtDestination
0,71121582,9901003
1,71122461,9901047
2,71122902,9901035
3,71146549,9901035
4,71147416,9901035
...,...,...
235375,131667110,9901035
235376,131801015,9901063
235377,131801460,9901005
235378,131801707,9901005


In [18]:
# Turn all categorical values into strings in the col_name to df dictionary
for key, dataframe in df_map.items():
    df_map[key] = dataframe.astype(str)


In [19]:
# Random check on the dict key(s)
df_map['CardiacRhythmOnArrivalAtDestination'].info()




<class 'pandas.core.frame.DataFrame'>
Index: 235380 entries, 0 to 235379
Data columns (total 2 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   PcrKey                               235380 non-null  object
 1   CardiacRhythmOnArrivalAtDestination  235380 non-null  object
dtypes: object(2)
memory usage: 5.4+ MB


In [21]:
df_copy = df.copy()



In [22]:
# Convert dtypes for string feature series and strip unnecessary spaces
feature_list_to_convert = ["PcrKey", "USCensusRegion", "USCensusDivision", "NasemsoRegion", "Urbanicity", 
                           "Gender", "Race", "ROSC", "EndOfEMS", "ResuscitationAttempted", 
                           "PriorUseOfAED", "ReasonToStopCPR", "ProtocolsUsed", "ArrestWitnessedBy", 
                           "TypeOfCPRProvided", "CardiacRhythmOnArrivalAtDestination"]




In [23]:
from sklearn.preprocessing import OneHotEncoder

def drop_values_from_df(df, col, values_to_drop):
    # Drop the specified values
    return df[~df[col].isin(values_to_drop)]

def one_hot_encode_and_group(df, key_column, categorical_column):
    # One-hot encode the categorical column
    ohe = OneHotEncoder(sparse_output=False, dtype=int)
    
    ohe_df = pd.DataFrame(ohe.fit_transform(df[[categorical_column]]),
                          columns=ohe.get_feature_names_out([categorical_column]))

    # Add the primary key to the one-hot encoded DataFrame
    ohe_df[key_column] = df[key_column].values

    # Group by primary key and use logical or to combine one-hot encoded values
    grouped = ohe_df.groupby(key_column).max().reset_index()
    return grouped





In [24]:
df_copy



,PcrKey,Age,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSystemResponseTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin,EMSTransportTimeMin,EMSTotalCallTimeMin,...,NasemsoRegion_Great Lakes,NasemsoRegion_South,NasemsoRegion_West,NasemsoRegion_Western Plains,EndOfEMS_Expired in ED,EndOfEMS_Expired in the Field,EndOfEMS_Ongoing Resuscitation in ED,EndOfEMS_ROSC in the Field,EndOfEMS_ROSC in the ED,EndOfEMS_Ongoing Resuscitation by Other EMS
0,71121582,23,11.000000,5.516667,34.933333,29.416667,25.556523,0.000000,14.676967,99.466667,...,0,1,0,0,0,1,0,0,0,0
1,71122461,120,0.000000,0.750000,4.133333,3.383333,18.316667,2.000000,6.733333,81.916667,...,0,1,0,0,0,0,1,0,0,0
2,71122902,63,0.000000,1.100000,8.683333,7.583333,34.650000,0.650000,15.416667,104.033333,...,0,1,0,0,1,0,0,0,0,0
3,71123389,92,114.000000,3.683333,7.366667,3.683333,14.083333,0.966667,4.212406,21.450000,...,0,1,0,0,0,1,0,0,0,0
4,71124833,48,88.760114,0.016667,9.133333,9.116667,23.081620,1.566667,8.590389,58.883333,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358858,131801464,57,0.000000,3.000000,8.000000,5.000000,13.000000,0.000000,2.612451,21.000000,...,1,0,0,0,0,1,0,0,0,0
358859,131801585,61,0.000000,6.000000,12.000000,6.000000,5.000000,0.000000,27.000000,119.000000,...,1,0,0,0,0,0,1,0,0,0
358860,131801624,92,0.000000,3.000000,5.000000,2.000000,32.000000,1.000000,2.860706,37.000000,...,1,0,0,0,0,1,0,0,0,0
358861,131801707,85,0.000000,6.000000,12.000000,6.000000,9.000000,2.000000,42.000000,146.000000,...,1,0,0,0,1,0,0,0,0,0


In [5]:
# Qingchu testing for Region columns Dec 08, 2024 (1)
import pandas as pd
import numpy as np

df_copy = pd.read_csv('clustering_data/has_disposition_clustering_df.csv')



In [9]:
# Qingchu testing for Region columns Dec 08, 2024 (2)
df_copy['USCensusRegion_Northeast'].value_counts(dropna=False)



USCensusRegion_Northeast
0.0    162572
1.0     27023
Name: count, dtype: int64

In [25]:
hf.convert_type_to_string(df_copy, ['PcrKey'])
df_copy.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358863 entries, 0 to 358862
Data columns (total 55 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   PcrKey                                          358863 non-null  object 
 1   Age                                             358863 non-null  int64  
 2   EMSDispatchCenterTimeSec                        358863 non-null  float64
 3   EMSChuteTimeMin                                 358863 non-null  float64
 4   EMSSystemResponseTimeMin                        358863 non-null  float64
 5   EMSSceneResponseTimeMin                         358863 non-null  float64
 6   EMSSceneTimeMin                                 358863 non-null  float64
 7   EMSSceneToPatientTimeMin                        358863 non-null  float64
 8   EMSTransportTimeMin                             358863 non-null  float64
 9   EMSTotalCallTimeMin       

In [28]:
# Iterate over the categorical columns and merge back to the unique DataFrame
for col in df_map:
    if df_map[col][col].dtype == 'object':  # Assuming 'object' type columns are categorical
        grouped_df = one_hot_encode_and_group(df_map[col], "PcrKey", col)
        df_copy = df_copy.merge(grouped_df, on="PcrKey", how="left")

df_copy





,PcrKey,Age,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSystemResponseTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin,EMSTransportTimeMin,EMSTotalCallTimeMin,...,CardiacRhythmOnArrivalAtDestination_9901053,CardiacRhythmOnArrivalAtDestination_9901055,CardiacRhythmOnArrivalAtDestination_9901057,CardiacRhythmOnArrivalAtDestination_9901059,CardiacRhythmOnArrivalAtDestination_9901061,CardiacRhythmOnArrivalAtDestination_9901063,CardiacRhythmOnArrivalAtDestination_9901065,CardiacRhythmOnArrivalAtDestination_9901067,CardiacRhythmOnArrivalAtDestination_9901069,CardiacRhythmOnArrivalAtDestination_9901071
0,71121582,23,11.000000,5.516667,34.933333,29.416667,25.556523,0.000000,14.676967,99.466667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,71122461,120,0.000000,0.750000,4.133333,3.383333,18.316667,2.000000,6.733333,81.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71122902,63,0.000000,1.100000,8.683333,7.583333,34.650000,0.650000,15.416667,104.033333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71123389,92,114.000000,3.683333,7.366667,3.683333,14.083333,0.966667,4.212406,21.450000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71124833,48,88.760114,0.016667,9.133333,9.116667,23.081620,1.566667,8.590389,58.883333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358858,131801464,57,0.000000,3.000000,8.000000,5.000000,13.000000,0.000000,2.612451,21.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358859,131801585,61,0.000000,6.000000,12.000000,6.000000,5.000000,0.000000,27.000000,119.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358860,131801624,92,0.000000,3.000000,5.000000,2.000000,32.000000,1.000000,2.860706,37.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358861,131801707,85,0.000000,6.000000,12.000000,6.000000,9.000000,2.000000,42.000000,146.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Impute one-hot-encoded treatment column data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def impute_numerical_data_mice(df, numerical_columns):
    imputer = IterativeImputer(max_iter=10, random_state=0)
    
    # Perform imputation
    imputed_data = imputer.fit_transform(df[numerical_columns])
    
    # Ensure no negative values and Age is rounded to the nearest integer
    imputed_data_df = pd.DataFrame(imputed_data, columns=numerical_columns)
    imputed_data_df = imputed_data_df.clip(lower=0)  # Ensure no negative values
    imputed_data_df['Age'] = imputed_data_df['Age'].round().astype(int)  # Round age to nearest integer
    
    return imputed_data_df

# df_onehot_imputed = impute_numerical_data_mice(df_copy, list(selected_features_for_imputation))




In [30]:
df_copy.columns


Index(['PcrKey', 'Age', 'EMSDispatchCenterTimeSec', 'EMSChuteTimeMin',
       'EMSSystemResponseTimeMin', 'EMSSceneResponseTimeMin',
       'EMSSceneTimeMin', 'EMSSceneToPatientTimeMin', 'EMSTransportTimeMin',
       'EMSTotalCallTimeMin',
       ...
       'CardiacRhythmOnArrivalAtDestination_9901053',
       'CardiacRhythmOnArrivalAtDestination_9901055',
       'CardiacRhythmOnArrivalAtDestination_9901057',
       'CardiacRhythmOnArrivalAtDestination_9901059',
       'CardiacRhythmOnArrivalAtDestination_9901061',
       'CardiacRhythmOnArrivalAtDestination_9901063',
       'CardiacRhythmOnArrivalAtDestination_9901065',
       'CardiacRhythmOnArrivalAtDestination_9901067',
       'CardiacRhythmOnArrivalAtDestination_9901069',
       'CardiacRhythmOnArrivalAtDestination_9901071'],
      dtype='object', length=137)

In [32]:
# Impute onehot treatment features newly added

# selected_features = ["ResuscitationAttempted", "PriorUseOfAED", "ReasonToStopCPR", "ProtocolsUsed", "ArrestWitnessedBy", 
#                      "TypeOfCPRProvided", "CardiacRhythmOnArrivalAtDestination"]

selected_features_for_imputation = set(df_copy.columns) - {"PcrKey"}
selected_features_for_imputation




{'Age',
 'ArrestRhythm_3011001',
 'ArrestRhythm_3011005',
 'ArrestRhythm_3011007',
 'ArrestRhythm_3011009',
 'ArrestRhythm_3011011',
 'ArrestRhythm_3011013',
 'ArrestRhythm_Unknown   ',
 'ArrestWitnessedBy_3004001',
 'ArrestWitnessedBy_3004003',
 'ArrestWitnessedBy_3004005',
 'ArrestWitnessedBy_3004007',
 'CardiacRhythmOnArrivalAtDestination_9901001',
 'CardiacRhythmOnArrivalAtDestination_9901003',
 'CardiacRhythmOnArrivalAtDestination_9901005',
 'CardiacRhythmOnArrivalAtDestination_9901007',
 'CardiacRhythmOnArrivalAtDestination_9901009',
 'CardiacRhythmOnArrivalAtDestination_9901011',
 'CardiacRhythmOnArrivalAtDestination_9901013',
 'CardiacRhythmOnArrivalAtDestination_9901015',
 'CardiacRhythmOnArrivalAtDestination_9901017',
 'CardiacRhythmOnArrivalAtDestination_9901019',
 'CardiacRhythmOnArrivalAtDestination_9901021',
 'CardiacRhythmOnArrivalAtDestination_9901023',
 'CardiacRhythmOnArrivalAtDestination_9901025',
 'CardiacRhythmOnArrivalAtDestination_9901027',
 'CardiacRhythmOnArriv

In [35]:
continuous_cols = ['Age', 'EMSDispatchCenterTimeSec', 'EMSChuteTimeMin', 'EMSSystemResponseTimeMin', 'EMSSceneResponseTimeMin', 
                   'EMSSceneTimeMin', 'EMSSceneToPatientTimeMin', 'EMSTransportTimeMin','EMSTotalCallTimeMin']
onehot_encoded_cols = list(set(selected_features_for_imputation) - set(continuous_cols))




In [36]:
# Simple Data Imputation for newly-appended columns (conditions / treatments)
from sklearn.impute import SimpleImputer

# Create a copy of the dataframe to apply imputation
df_imputed = df_copy.copy()

# Impute continuous features with mean
mean_imputer = SimpleImputer(strategy='mean')
df_imputed[continuous_cols] = mean_imputer.fit_transform(df_imputed[continuous_cols])

# Impute one-hot encoded features with mode
mode_imputer = SimpleImputer(strategy='most_frequent')
df_imputed[onehot_encoded_cols] = mode_imputer.fit_transform(df_imputed[onehot_encoded_cols])

# Make sure 'Age' is of integer type after mean imputation
df_imputed['Age'] = df_imputed['Age'].round().astype(int)




In [37]:
df_imputed




,PcrKey,Age,EMSDispatchCenterTimeSec,EMSChuteTimeMin,EMSSystemResponseTimeMin,EMSSceneResponseTimeMin,EMSSceneTimeMin,EMSSceneToPatientTimeMin,EMSTransportTimeMin,EMSTotalCallTimeMin,...,CardiacRhythmOnArrivalAtDestination_9901053,CardiacRhythmOnArrivalAtDestination_9901055,CardiacRhythmOnArrivalAtDestination_9901057,CardiacRhythmOnArrivalAtDestination_9901059,CardiacRhythmOnArrivalAtDestination_9901061,CardiacRhythmOnArrivalAtDestination_9901063,CardiacRhythmOnArrivalAtDestination_9901065,CardiacRhythmOnArrivalAtDestination_9901067,CardiacRhythmOnArrivalAtDestination_9901069,CardiacRhythmOnArrivalAtDestination_9901071
0,71121582,23,11.000000,5.516667,34.933333,29.416667,25.556523,0.000000,14.676967,99.466667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,71122461,120,0.000000,0.750000,4.133333,3.383333,18.316667,2.000000,6.733333,81.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71122902,63,0.000000,1.100000,8.683333,7.583333,34.650000,0.650000,15.416667,104.033333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71123389,92,114.000000,3.683333,7.366667,3.683333,14.083333,0.966667,4.212406,21.450000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,71124833,48,88.760114,0.016667,9.133333,9.116667,23.081620,1.566667,8.590389,58.883333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358858,131801464,57,0.000000,3.000000,8.000000,5.000000,13.000000,0.000000,2.612451,21.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358859,131801585,61,0.000000,6.000000,12.000000,6.000000,5.000000,0.000000,27.000000,119.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358860,131801624,92,0.000000,3.000000,5.000000,2.000000,32.000000,1.000000,2.860706,37.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358861,131801707,85,0.000000,6.000000,12.000000,6.000000,9.000000,2.000000,42.000000,146.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# Save df into csv
df_imputed.to_csv("./clustering_data/imputed_background_treatment_data.csv", index = False)


